# The Battle of Neighborhoods

# Looking for places in England to start a Bandeoke business

Importing necessary libraries

In [1]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import json
import pandas as pd
import requests
from bs4 import BeautifulSoup
import geocoder

In [3]:
from builtins import any as b_any

In [4]:
!conda install -c conda-forge geopy --yes 

Solving environment: done

# All requested packages already installed.



In [5]:
from geopy.geocoders import Nominatim

In [6]:
import requests
from pandas.io.json import json_normalize

In [7]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [8]:
from sklearn.cluster import KMeans

In [9]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

# All requested packages already installed.



# 1. Data acquisition

## 1.1. Bands dataframe

### Bands by origin and URL bands 

In [10]:
url_bands='https://en.wikipedia.org/wiki/List_of_music_artists_and_bands_from_England'
wiki_url_bands = requests.get(url_bands).text

In [11]:
soup = BeautifulSoup(wiki_url_bands,'lxml')
#soup.prettify()

In [12]:
bands=soup.select('ul li a')
names = [band.get('title') for band in bands]
refs = [band.get('href') for band in bands]

In [13]:
bands_df=pd.DataFrame()
bands_df['Band']=names
bands_df['Link']=refs
bands_df.dropna(inplace=True)

In [14]:
locations = soup.select('ul li')
cities_aux = [loc.get_text().split('\n') for loc in locations]
cities = [i[0:-1] for i in cities_aux if len(i)>1]
cities = [np.repeat(i[0],len(i)).tolist() for i in cities]
cit=[]
cities=[cit+i for i in cities]
for i in cities:
    cit=cit+i
cities=cit

In [15]:
cities[len(cities):len(bands_df)]=[np.nan for i in range(len(bands_df)-len(cities))]

In [16]:
bands_df['City']=cities

In [17]:
bands_df.dropna(inplace=True)
bands_df.reset_index(inplace=True,drop=True)

In [18]:
bands_df=bands_df.iloc[0:996]

In [19]:
bands_df.replace('/wiki/Hadouken!', '/wiki/Hadouken!_(band)',inplace=True)

In [20]:
bands_df.head(10)

,Band,Link,City
0,Radiohead,/wiki/Radiohead,Abingdon
1,The Troggs,/wiki/The_Troggs,Andover
2,The Young Knives,/wiki/The_Young_Knives,Ashby-de-la-Zouch
3,Marillion,/wiki/Marillion,Aylesbury
4,The Danse Society,/wiki/The_Danse_Society,Barnsley
5,Saxon (band),/wiki/Saxon_(band),Barnsley
6,The Ward Brothers,/wiki/The_Ward_Brothers,Barnsley
7,The Assembly,/wiki/The_Assembly,Basildon
8,Depeche Mode,/wiki/Depeche_Mode,Basildon
9,Yazoo (band),/wiki/Yazoo_(band),Basildon


### Musical genres played by band

In [21]:
i=0
for link in bands_df['Link']:
    url_band='https://en.wikipedia.org'+link
    wiki_band=requests.get(url_band).text
    soup_band = BeautifulSoup(wiki_band,'lxml')
    avail=soup_band.find_all(scope='row')[0:15]
    avail=[data.get_text() for data in avail]
    if ('Genres' in avail):
        data_band=[data.get_text() for data in soup_band.select('tr td')[0:12]]
        indice = [i for i, s in enumerate(data_band) if ((('Rock' in s)|('music' in s)|('rock' in s)|('pop' in s)|('punk' in s)|('wave' in s))&~('Wiki' in s))]
        if indice:
            genres=[gen.get('title') for gen in soup_band.select('tr td')[indice[0]].select('a')]
            genres = [x for x in genres if x]
            try: 
                bands_df.loc[i,'Genre1']=genres[0]
            except:
                bands_df.loc[i,'Genre1']=np.nan                
            try: 
                bands_df.loc[i,'Genre2']=genres[1]
            except:
                bands_df.loc[i,'Genre2']=np.nan
            try: 
                bands_df.loc[i,'Genre3']=genres[2]
            except:
                bands_df.loc[i,'Genre3']=np.nan
            try: 
                bands_df.loc[i,'Genre4']=genres[3]
            except:
                bands_df.loc[i,'Genre4']=np.nan
        else:
            indice = [i for i, s in enumerate(data_band) if ((', England' in s)|(', UK' in s)|(', United Kingdom' in s))]
            if indice:
                genres=[gen.get('title') for gen in soup_band.select('tr td')[indice[0]+1].select('a')]
                genres = [x for x in genres if x]
                try:   
                    bands_df.loc[i,'Genre1']=genres[0]
                except:
                    bands_df.loc[i,'Genre1']=np.nan
                try:   
                    bands_df.loc[i,'Genre2']=genres[1]
                except:
                    bands_df.loc[i,'Genre2']=np.nan
                try:   
                    bands_df.loc[i,'Genre3']=genres[2]
                except:
                    bands_df.loc[i,'Genre3']=np.nan
                try:   
                    bands_df.loc[i,'Genre4']=genres[3]
                except:
                    bands_df.loc[i,'Genre4']=np.nan 
                    
    i=i+1

Making some editions

In [22]:
bands_df.dropna(subset=['Genre1'],inplace=True)
bands_df.reset_index(drop=True,inplace=True)
bands_df.loc[bands_df[bands_df['Band']=='Killing Joke'].index.values,['Genre1','Genre2','Genre3','Genre4']]=['POST-PUNK','INDUSTRIAL ROCK','GOTHIC ROCK','NEW WAVE']

In [23]:
bands_df['Genre1']=bands_df['Genre1'].str.upper() 
bands_df['Genre2']=bands_df['Genre2'].str.upper() 
bands_df['Genre3']=bands_df['Genre3'].str.upper() 
bands_df['Genre4']=bands_df['Genre4'].str.upper() 

In [24]:
dict_genres={'2 TONE (MUSIC GENRE)':'2 TONE',
            'AMERICANA (MUSIC)':'AMERICANA',
            'CHAMBER ROCK (PAGE DOES NOT EXIST)':'CHAMBER ROCK',
            'ELECTRO (MUSIC)':'ELECTRO',
             'ROCK AND ROLL':"ROCK 'N ROLL",
            'GARAGE PUNK (FUSION GENRE)':'GARAGE PUNK',
            'GARAGE PUNK ROCK (PAGE DOES NOT EXIST)':'GARAGE PUNK',
            'GEORGIA (U.S. STATE)':'GEORGIA',
            'GLITCH (MUSIC)':'GLITCH',
            'GOSPEL MUSIC':'GOSPEL',
            'GREBO (MUSIC)':'GREBO',
            'GRIME (MUSIC GENRE)':'GRIME',
            'GRIME (MUSIC)':'GRIME',
            'GRIME MUSIC':'GRIME',
            'HEAVY METAL MUSIC':'HEAVY METAL',
            'HIP HOP MUSIC':'HIP HOP',
            'INDUSTRIAL MUSIC':'INDUSTRIAL',
            'JAZZ MUSIC':'JAZZ',
            'LO-FI MUSIC':'LO-FI',
            'MERSEYBEAT':'MERSEY BEAT',
            'NEW WAVE MUSIC':'NEW WAVE',
            'NEW-AGE MUSIC':'NEW-AGE',
            'POP (MUSIC)':'POP MUSIC',
            'POP PUNK MUSIC':'POP PUNK',
            'POST PUNK':'POST-PUNK',
            'POST-INDUSTRIAL MUSIC':'POST-INDUSTRIAL',
            'POST-PUNK REVIVAL':'POST-PUNK REVIVAL',
            'PUNK MUSIC':'PUNK',
            'ROCK (MUSIC)':'ROCK',
            'ROCK MUSIC':'ROCK',
            'SURF PUNK (MUSIC GENRE)':'SURF PUNK',
            'TWO-TONE (MUSIC GENRE)':'2-TONE',
             '2 TONE':'2-TONE',
             'SYNTH-POP':'SYNTHPOP'
            }

In [25]:
for i in list(dict_genres.keys()):
    bands_df[['Genre1','Genre2','Genre3','Genre4']]=bands_df[['Genre1','Genre2','Genre3','Genre4']].replace(i,dict_genres[i])

In [26]:
bands_df.head(10)

,Band,Link,City,Genre1,Genre2,Genre3,Genre4
0,Radiohead,/wiki/Radiohead,Abingdon,ART ROCK,ALTERNATIVE ROCK,ELECTRONICA,EXPERIMENTAL ROCK
1,The Troggs,/wiki/The_Troggs,Andover,GARAGE ROCK,POP ROCK,PROTO-PUNK,NaN
2,The Young Knives,/wiki/The_Young_Knives,Ashby-de-la-Zouch,POST-PUNK,ART PUNK,EXPERIMENTAL ROCK,INDIE ROCK
3,Marillion,/wiki/Marillion,Aylesbury,NEO-PROGRESSIVE ROCK,SYMPHONIC ROCK,ART ROCK,POST-PUNK
4,The Danse Society,/wiki/The_Danse_Society,Barnsley,GOTHIC ROCK,POST-PUNK,DARK WAVE,SYNTHPOP
5,Saxon (band),/wiki/Saxon_(band),Barnsley,HEAVY METAL,NaN,NaN,NaN
6,The Ward Brothers,/wiki/The_Ward_Brothers,Barnsley,POP ROCK,NEW WAVE,NaN,NaN
7,Depeche Mode,/wiki/Depeche_Mode,Basildon,SYNTHPOP,NEW WAVE,DANCE-ROCK,ELECTRONIC ROCK
8,Yazoo (band),/wiki/Yazoo_(band),Basildon,SYNTHPOP,NEW WAVE,NaN,NaN
9,Goldfrapp,/wiki/Goldfrapp,Bath,SYNTHPOP,ELECTROCLASH,TRIP HOP,FOLKTRONICA


### Grouping the genres into supergenres

In [27]:
musicmap=requests.get('https://musicmap.info/').text
soup_musicmap = BeautifulSoup(musicmap,'lxml')
#soup_musicmap.prettify()

In [28]:
def get_sub_titles(soup):
    print(soup.get('id'))
    list_subs=[sub.get_text() for sub in soup.select('text')]
    list_subs=[sub.replace("\t", "") for sub in list_subs]
    list_subs=[sub[1:-1].replace("\n", " ") for sub in list_subs]
    list_subs=[sub.replace("- ", "-") for sub in list_subs]
    return list_subs

In [29]:
titles_alt=['JANGLE POP','INDIE ROCK','NOISE ROCK','DREAM POP',
           'SHOEGAZE','SHOEGAZING','GRUNGE','RAP ROCK', 'RAPCORE', 'FUNK METAL',
           'SKATE PUNK','POP PUNK','MATH ROCK', 'MATHCORE','ALTERNATIVE ROCK','INDIE II',
           'BRITPOP','POST-GRUNGE','POST-ROCK']
titles_alt

['JANGLE POP',
 'INDIE ROCK',
 'NOISE ROCK',
 'DREAM POP',
 'SHOEGAZE',
 'SHOEGAZING',
 'GRUNGE',
 'RAP ROCK',
 'RAPCORE',
 'FUNK METAL',
 'SKATE PUNK',
 'POP PUNK',
 'MATH ROCK',
 'MATHCORE',
 'ALTERNATIVE ROCK',
 'INDIE II',
 'BRITPOP',
 'POST-GRUNGE',
 'POST-ROCK']

In [30]:
titles_hcp=['ANARCHO-PUNK', 'CRUST PUNK', 'D-BEAT / DISCORE','HARDCORE PUNK',
           'CROSSOVER THRASH','GRINDCORE','POST-HARDCORE','EMO(CORE) & SCREAMO',
            'MATH ROCK','MATHCORE','METALCORE','NWOAHM',
           'SYNTHCORE', 'CRUNKCORE']
titles_hcp

['ANARCHO-PUNK',
 'CRUST PUNK',
 'D-BEAT / DISCORE',
 'HARDCORE PUNK',
 'CROSSOVER THRASH',
 'GRINDCORE',
 'POST-HARDCORE',
 'EMO(CORE) & SCREAMO',
 'MATH ROCK',
 'MATHCORE',
 'METALCORE',
 'NWOAHM',
 'SYNTHCORE',
 'CRUNKCORE']

In [31]:
titles_pwv=['PUB ROCK','PROTO PUNK','PROTO-PUNK','PUNK ROCK','ANARCHO-PUNK', 'CRUST PUNK','D-BEAT / DISCORE',
           'SKA REVIVAL', '2-TONE','SKA PUNK','SKACORE','POST-PUNK','NO WAVE','NEW WAVE','ELECTROCLASH',
           'HORROR PUNK','PSYCHOBILLY','SYNTHPOP','ELECTROPOP','NEW ROMANTICS','SKATE PUNK', 'POP PUNK']
titles_pwv

['PUB ROCK',
 'PROTO PUNK',
 'PROTO-PUNK',
 'PUNK ROCK',
 'ANARCHO-PUNK',
 'CRUST PUNK',
 'D-BEAT / DISCORE',
 'SKA REVIVAL',
 '2-TONE',
 'SKA PUNK',
 'SKACORE',
 'POST-PUNK',
 'NO WAVE',
 'NEW WAVE',
 'ELECTROCLASH',
 'HORROR PUNK',
 'PSYCHOBILLY',
 'SYNTHPOP',
 'ELECTROPOP',
 'NEW ROMANTICS',
 'SKATE PUNK',
 'POP PUNK']

In [32]:
titles_gld=['MERSEY BEAT','FOLK ROCK','BRITISH BLUES', 'BLUES ROCK',
           'PSYCHEDELIC ROCK', 'ACID ROCK','PSYCHEDELIA','HARD ROCK',
            'PROGRESSIVE ROCK', 'ART ROCK','SYMPHONIC ROCK','KRAUTROCK',
           'GLAM ROCK','GLITTER ROCK','SHOCK ROCK','SOUTHERN ROCK','ROCK',
           'HEARTLAND ROCK', 'A.O.R. (ADULT ORIENTED ROCK)']
titles_gld

['MERSEY BEAT',
 'FOLK ROCK',
 'BRITISH BLUES',
 'BLUES ROCK',
 'PSYCHEDELIC ROCK',
 'ACID ROCK',
 'PSYCHEDELIA',
 'HARD ROCK',
 'PROGRESSIVE ROCK',
 'ART ROCK',
 'SYMPHONIC ROCK',
 'KRAUTROCK',
 'GLAM ROCK',
 'GLITTER ROCK',
 'SHOCK ROCK',
 'SOUTHERN ROCK',
 'ROCK',
 'HEARTLAND ROCK',
 'A.O.R. (ADULT ORIENTED ROCK)']

In [33]:
titles_rnr=['SKIFFLE (REVIVAL)',"ROCK 'N ROLL", 'ROCKABILLY','AMERICAN FOLK REVIVAL', 'BRITISH FOLK REVIVAL',
           'SURF ROCK','INSTRUMENTAL ROCK','GARAGE ROCK']
titles_rnr

['SKIFFLE (REVIVAL)',
 "ROCK 'N ROLL",
 'ROCKABILLY',
 'AMERICAN FOLK REVIVAL',
 'BRITISH FOLK REVIVAL',
 'SURF ROCK',
 'INSTRUMENTAL ROCK',
 'GARAGE ROCK']

In [34]:
titles_con=['POST-BRITPOP','EMO-ROCK','EMO','GARAGE REVIVAL','POST-PUNK REVIVAL','NU-RAWK',
           'NEW PROG','NU PROG', 'POST PROG','INDIETRONICA', 'CHILLWAVE',
           'DANCE-PUNK','NU RAVE','INDIE FOLK','FREAKFOLK','NEW WEIRD AMERICA']
titles_con

['POST-BRITPOP',
 'EMO-ROCK',
 'EMO',
 'GARAGE REVIVAL',
 'POST-PUNK REVIVAL',
 'NU-RAWK',
 'NEW PROG',
 'NU PROG',
 'POST PROG',
 'INDIETRONICA',
 'CHILLWAVE',
 'DANCE-PUNK',
 'NU RAVE',
 'INDIE FOLK',
 'FREAKFOLK',
 'NEW WEIRD AMERICA']

In [35]:
titles_met=['CLASSIC METAL', 'HEAVY METAL','PROGRESSIVE METAL','POWER METAL','GLAM METAL','HAIR METAL','POP METAL',
           'NWOBHM','THRASH METAL','EXTREME METAL','BLACK I METAL','SPEED METAL',
           'CROSSOVER THRASH','DOOM METAL','DEATH METAL','GRINDCORE',
            'STONER METAL', 'SLUDGE METAL','ROCK METAL','SYMPHONIC METAL','GOTHIC METAL',
           'BLACK METAL','NU METAL','RAP METAL','METALCORE','NWOAHM']
titles_met

['CLASSIC METAL',
 'HEAVY METAL',
 'PROGRESSIVE METAL',
 'POWER METAL',
 'GLAM METAL',
 'HAIR METAL',
 'POP METAL',
 'NWOBHM',
 'THRASH METAL',
 'EXTREME METAL',
 'BLACK I METAL',
 'SPEED METAL',
 'CROSSOVER THRASH',
 'DOOM METAL',
 'DEATH METAL',
 'GRINDCORE',
 'STONER METAL',
 'SLUDGE METAL',
 'ROCK METAL',
 'SYMPHONIC METAL',
 'GOTHIC METAL',
 'BLACK METAL',
 'NU METAL',
 'RAP METAL',
 'METALCORE',
 'NWOAHM']

In [36]:
titles_inr=['KRAUTROCK','AVANT-GARDE INDUSTRIAL','GOTHIC ROCK','DEATHROCK',
           'ELECTRONIC BODY MUSIC','NOISE MUSIC','DARK AMBIENT', 'DARK INDUSTRIAL',
           'DARK WAVE','COLDWAVE','INDUSTRIAL ROCK','INDUSTRIAL METAL',
           'ELECTRO-INDUSTRIAL','AGGREPPO','FUTUREPOP',
            'MINIMAL WAVE','SYNTH & MINIMAL INDUSTRIAL (REVIVAL)']           
titles_inr

['KRAUTROCK',
 'AVANT-GARDE INDUSTRIAL',
 'GOTHIC ROCK',
 'DEATHROCK',
 'ELECTRONIC BODY MUSIC',
 'NOISE MUSIC',
 'DARK AMBIENT',
 'DARK INDUSTRIAL',
 'DARK WAVE',
 'COLDWAVE',
 'INDUSTRIAL ROCK',
 'INDUSTRIAL METAL',
 'ELECTRO-INDUSTRIAL',
 'AGGREPPO',
 'FUTUREPOP',
 'MINIMAL WAVE',
 'SYNTH & MINIMAL INDUSTRIAL (REVIVAL)']

In [37]:
super_genres=['INDUSTRIAL & GOTHIC','HEAVY METAL',
              "ROCK 'N' ROLL",'GOLDEN AGE / CLASSIC ROCK','PUNK ROCK / NEW WAVE',
              'HARDCORE PUNK','ALTERNATIVE ROCK / INDIE','CONTEMPORARY ROCK']
genres=[titles_inr,titles_met,titles_rnr,titles_gld,titles_pwv,titles_hcp,titles_alt,titles_con]
genres_df=pd.DataFrame({'Super-genres':super_genres,'Genres':genres})

In [38]:
genres_df

,Super-genres,Genres
0,INDUSTRIAL & GOTHIC,"[KRAUTROCK, AVANT-GARDE INDUSTRIAL, GOTHIC ROC..."
1,HEAVY METAL,"[CLASSIC METAL, HEAVY METAL, PROGRESSIVE METAL..."
2,ROCK 'N' ROLL,"[SKIFFLE (REVIVAL), ROCK 'N ROLL, ROCKABILLY, ..."
3,GOLDEN AGE / CLASSIC ROCK,"[MERSEY BEAT, FOLK ROCK, BRITISH BLUES, BLUES ..."
4,PUNK ROCK / NEW WAVE,"[PUB ROCK, PROTO PUNK, PROTO-PUNK, PUNK ROCK, ..."
5,HARDCORE PUNK,"[ANARCHO-PUNK, CRUST PUNK, D-BEAT / DISCORE, H..."
6,ALTERNATIVE ROCK / INDIE,"[JANGLE POP, INDIE ROCK, NOISE ROCK, DREAM POP..."
7,CONTEMPORARY ROCK,"[POST-BRITPOP, EMO-ROCK, EMO, GARAGE REVIVAL, ..."


In [39]:
def get_super_genres(gen_col):
    sg=[np.nan]*len(gen_col)
    for i,gen in enumerate(gen_col):
        if gen!=np.nan:
            for j in range(8):
                if gen in genres_df.loc[j,'Genres']:                        
                    sg[i]=genres_df.loc[j,'Super-genres']
    return sg

In [40]:
bands_df['SG1']=get_super_genres(bands_df['Genre1'])
bands_df['SG2']=get_super_genres(bands_df['Genre2'])
bands_df['SG3']=get_super_genres(bands_df['Genre3'])
bands_df['SG4']=get_super_genres(bands_df['Genre4'])

In [41]:
bands_df['COUNT']=bands_df[['SG1','SG2','SG3','SG4']].count(axis=1)
bands_df=bands_df[bands_df['COUNT']>0]

In [42]:
bands_df.drop(['COUNT'],axis=1,inplace=True)

In [43]:
bands_df.reset_index(inplace=True,drop=True)
bands_df[['Band','Genre1','Genre2','Genre3','Genre4','SG1','SG2','SG3','SG4']].tail(10)

,Band,Genre1,Genre2,Genre3,Genre4,SG1,SG2,SG3,SG4
711,The Ordinary Boys,INDIE ROCK,PUNK ROCK,SKA,2-TONE,ALTERNATIVE ROCK / INDIE,PUNK ROCK / NEW WAVE,NaN,PUNK ROCK / NEW WAVE
712,Chris Helme,FOLK MUSIC,ALTERNATIVE ROCK,BRITPOP,NaN,NaN,ALTERNATIVE ROCK / INDIE,ALTERNATIVE ROCK / INDIE,NaN
713,The Batfish Boys,GOTHIC ROCK,GOTHABILLY,NaN,NaN,INDUSTRIAL & GOTHIC,NaN,NaN,NaN
714,The Redskins,PUNK ROCK,SOUL MUSIC,ROCKABILLY,POP MUSIC,PUNK ROCK / NEW WAVE,NaN,ROCK 'N' ROLL,NaN
715,Shed Seven,ALTERNATIVE ROCK,BRITPOP,NaN,NaN,ALTERNATIVE ROCK / INDIE,ALTERNATIVE ROCK / INDIE,NaN,NaN
716,The Seahorses,ALTERNATIVE ROCK,INDIE ROCK,BRITPOP,NaN,ALTERNATIVE ROCK / INDIE,ALTERNATIVE ROCK / INDIE,ALTERNATIVE ROCK / INDIE,NaN
717,The Smoke,PSYCHEDELIC ROCK,FREAKBEAT,PSYCHEDELIC POP,NaN,GOLDEN AGE / CLASSIC ROCK,NaN,NaN,NaN
718,Glamour of the Kill,POST-HARDCORE,METALCORE,HEAVY METAL,NaN,HARDCORE PUNK,HARDCORE PUNK,HEAVY METAL,NaN
719,Elliot Minor,SYMPHONIC ROCK,POP ROCK,CLASSICAL MUSIC,NaN,GOLDEN AGE / CLASSIC ROCK,NaN,NaN,NaN
720,RSJ (band),METALCORE,MATHCORE,NaN,NaN,HARDCORE PUNK,ALTERNATIVE ROCK / INDIE,NaN,NaN


In [44]:
SG1=pd.get_dummies(bands_df['SG1'])
SG2=pd.get_dummies(bands_df['SG2'])
SG3=pd.get_dummies(bands_df['SG3'])
SG4=pd.get_dummies(bands_df['SG4'])

In [45]:
bands_df=bands_df.join(SG1|SG2|SG3|SG4)

In [46]:
bands_df.drop(['SG1','SG2','SG3','SG4'],axis=1,inplace=True)

In [47]:
bands_df['City'].replace('Abingdon','Abingdon-on-Thames',inplace=True)
bands_df['City'].replace('Bexhill on Sea','Bexhill-on-Sea',inplace=True)
bands_df['City'].replace('Leamington Spa','Royal Leamington Spa',inplace=True)
bands_df['City'].replace('Salisbury, Wiltshire','Salisbury',inplace=True)
bands_df['City'].replace('Southend','Southend-on-Sea',inplace=True)

In [48]:
bands_df.drop(['Link','Genre1','Genre2','Genre3','Genre4'],axis=1,inplace=True)

In [49]:
bands_df.tail(10)

,Band,City,ALTERNATIVE ROCK / INDIE,CONTEMPORARY ROCK,GOLDEN AGE / CLASSIC ROCK,HARDCORE PUNK,HEAVY METAL,INDUSTRIAL & GOTHIC,PUNK ROCK / NEW WAVE,ROCK 'N' ROLL
711,The Ordinary Boys,Worthing,1,0,0,0,0,0,1,0
712,Chris Helme,York,1,0,0,0,0,0,0,0
713,The Batfish Boys,York,0,0,0,0,0,1,0,0
714,The Redskins,York,0,0,0,0,0,0,1,1
715,Shed Seven,York,1,0,0,0,0,0,0,0
716,The Seahorses,York,1,0,0,0,0,0,0,0
717,The Smoke,York,0,0,1,0,0,0,0,0
718,Glamour of the Kill,York,0,0,0,1,1,0,0,0
719,Elliot Minor,York,0,0,1,0,0,0,0,0
720,RSJ (band),York,1,0,0,1,0,0,0,0


Create another group where 'PUNK ROCK / NEW WAVE' and 'ALTERNATIVE ROCK / INDIE' are included

In [53]:
bands_df['UNDERGROUND']=bands_df['ALTERNATIVE ROCK / INDIE']|bands_df['PUNK ROCK / NEW WAVE']
bands_df.tail(10)

,Band,City,ALTERNATIVE ROCK / INDIE,CONTEMPORARY ROCK,GOLDEN AGE / CLASSIC ROCK,HARDCORE PUNK,HEAVY METAL,INDUSTRIAL & GOTHIC,PUNK ROCK / NEW WAVE,ROCK 'N' ROLL,UNDERGROUND
711,The Ordinary Boys,Worthing,1,0,0,0,0,0,1,0,1
712,Chris Helme,York,1,0,0,0,0,0,0,0,1
713,The Batfish Boys,York,0,0,0,0,0,1,0,0,0
714,The Redskins,York,0,0,0,0,0,0,1,1,1
715,Shed Seven,York,1,0,0,0,0,0,0,0,1
716,The Seahorses,York,1,0,0,0,0,0,0,0,1
717,The Smoke,York,0,0,1,0,0,0,0,0,0
718,Glamour of the Kill,York,0,0,0,1,1,0,0,0,0
719,Elliot Minor,York,0,0,1,0,0,0,0,0,0
720,RSJ (band),York,1,0,0,1,0,0,0,0,1


In [66]:
resume_cities=bands_df.groupby('City').sum().sort_values('UNDERGROUND',ascending=False)
resume_cities

,ALTERNATIVE ROCK / INDIE,CONTEMPORARY ROCK,GOLDEN AGE / CLASSIC ROCK,HARDCORE PUNK,HEAVY METAL,INDUSTRIAL & GOTHIC,PUNK ROCK / NEW WAVE,ROCK 'N' ROLL,UNDERGROUND
City,,,,,,,,,
London,96,26,69,8,16,8,72,13,150
Manchester,55,9,16,2,1,3,31,2,71
Leeds,23,6,2,5,1,5,11,0,29
Liverpool,19,2,6,1,1,0,11,2,25
Sheffield,16,4,6,2,1,1,11,4,24
Brighton,13,6,5,3,0,0,3,2,15
Birmingham,6,0,8,4,6,1,11,0,14
Leicester,10,1,7,1,1,0,3,1,12
Surrey,5,2,5,3,0,0,2,0,7


### Analysis by density 

In [123]:
pop = pd.read_excel('/Users/mamakino/Downloads/Capstone/Population.xlsx')

In [124]:
resume_cities_dens=bands_df.groupby('City').sum().sort_values('UNDERGROUND',ascending=False)
resume_cities_dens.head()

,ALTERNATIVE ROCK / INDIE,CONTEMPORARY ROCK,GOLDEN AGE / CLASSIC ROCK,HARDCORE PUNK,HEAVY METAL,INDUSTRIAL & GOTHIC,PUNK ROCK / NEW WAVE,ROCK 'N' ROLL,UNDERGROUND
City,,,,,,,,,
London,96,26,69,8,16,8,72,13,150
Manchester,55,9,16,2,1,3,31,2,71
Leeds,23,6,2,5,1,5,11,0,29
Liverpool,19,2,6,1,1,0,11,2,25
Sheffield,16,4,6,2,1,1,11,4,24


In [125]:
for cit in resume_cities_dens.index.tolist():
    try:
        resume_cities_dens.loc[cit,'Population']=pop.loc[int(pop[pop['City/Town']==cit].index.values)]['Population']
    except:
        resume_cities_dens.loc[cit,'Population']==np.nan

In [126]:
resume_cities_dens.head()

,ALTERNATIVE ROCK / INDIE,CONTEMPORARY ROCK,GOLDEN AGE / CLASSIC ROCK,HARDCORE PUNK,HEAVY METAL,INDUSTRIAL & GOTHIC,PUNK ROCK / NEW WAVE,ROCK 'N' ROLL,UNDERGROUND,Population
City,,,,,,,,,,
London,96,26,69,8,16,8,72,13,150,8907918.0
Manchester,55,9,16,2,1,3,31,2,71,554400.0
Leeds,23,6,2,5,1,5,11,0,29,503388.0
Liverpool,19,2,6,1,1,0,11,2,25,579256.0
Sheffield,16,4,6,2,1,1,11,4,24,544402.0


In [127]:
resume_cities_dens.reset_index(inplace=True)

In [128]:
resume_cities_dens.dropna(inplace=True)

In [129]:
resume_cities_dens['ALTERNATIVE ROCK / INDIE']=resume_cities_dens['ALTERNATIVE ROCK / INDIE']/resume_cities_dens['Population']*1000
resume_cities_dens['CONTEMPORARY ROCK']=resume_cities_dens['CONTEMPORARY ROCK']/resume_cities_dens['Population']*1000
resume_cities_dens['GOLDEN AGE / CLASSIC ROCK']=resume_cities_dens['GOLDEN AGE / CLASSIC ROCK']/resume_cities_dens['Population']*1000
resume_cities_dens['HARDCORE PUNK']=resume_cities_dens['HARDCORE PUNK']/resume_cities_dens['Population']*1000
resume_cities_dens['HEAVY METAL']=resume_cities_dens['HEAVY METAL']/resume_cities_dens['Population']*1000
resume_cities_dens['INDUSTRIAL & GOTHIC']=resume_cities_dens['INDUSTRIAL & GOTHIC']/resume_cities_dens['Population']*1000
resume_cities_dens['PUNK ROCK / NEW WAVE']=resume_cities_dens['PUNK ROCK / NEW WAVE']/resume_cities_dens['Population']*1000
resume_cities_dens["ROCK 'N' ROLL"]=resume_cities_dens["ROCK 'N' ROLL"]/resume_cities_dens['Population']*1000
resume_cities_dens['UNDERGROUND']=resume_cities_dens['UNDERGROUND']/resume_cities_dens['Population']*1000

In [130]:
resume_cities_dens.sort_values('UNDERGROUND',ascending=False,inplace=True)

In [135]:
resume_cities_dens.reset_index(inplace=True,drop=True)
resume_cities_dens.head(10)

,City,ALTERNATIVE ROCK / INDIE,CONTEMPORARY ROCK,GOLDEN AGE / CLASSIC ROCK,HARDCORE PUNK,HEAVY METAL,INDUSTRIAL & GOTHIC,PUNK ROCK / NEW WAVE,ROCK 'N' ROLL,UNDERGROUND,Population
0,Manchester,0.099206,0.016234,0.028860,0.003608,0.001804,0.005411,0.055916,0.003608,0.128066,554400.0
1,Frodsham,0.108378,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.108378,9227.0
2,Ashby-de-la-Zouch,0.072448,0.000000,0.000000,0.000000,0.000000,0.000000,0.072448,0.000000,0.072448,13803.0
3,Beccles,0.070587,0.000000,0.070587,0.070587,0.000000,0.000000,0.000000,0.000000,0.070587,14167.0
4,Teignmouth,0.064115,0.000000,0.064115,0.000000,0.000000,0.000000,0.000000,0.000000,0.064115,15597.0
5,Matlock,0.063910,0.000000,0.063910,0.000000,0.000000,0.000000,0.000000,0.000000,0.063910,15647.0
6,Brighton,0.053719,0.024793,0.020661,0.012397,0.000000,0.000000,0.012397,0.008264,0.061984,241999.0
7,Leeds,0.045690,0.011919,0.003973,0.009933,0.001987,0.009933,0.021852,0.000000,0.057610,503388.0
8,Nantwich,0.055546,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055546,18003.0
9,Tewkesbury,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047902,0.000000,0.047902,20876.0


## 1.2. Exploring Manchester areas 

In [181]:
url_manch_areas='https://en.wikipedia.org/wiki/Category:Areas_of_Manchester'
wiki_manch_areas = requests.get(url_manch_areas).text
soup4 = BeautifulSoup(wiki_manch_areas,'lxml')

In [182]:
areas=soup4.select('ul li a')
names = [area.get('title') for area in areas]
refs = [area.get('href') for area in areas]

In [183]:
areas=pd.DataFrame()
areas['Neighborhood']=names
areas['Link']=refs
areas=areas.loc[2:70]

In [184]:
areas.reset_index(drop=True,inplace=True)

In [185]:
for i,link in enumerate(areas['Link']):
    url_area='https://en.wikipedia.org'+link
    wiki_area=requests.get(url_area).text
    soup_area = BeautifulSoup(wiki_area,'lxml')
    tags=soup_area.select('span a span span')
    for j in range(len(tags)):
        if tags[j].get('class')==['geo']:
            geo=j
            coord=tags[geo].get_text().split(';')
            areas.loc[i,'Latitude']=coord[0]
            areas.loc[i,'Longitude']=coord[1]

In [186]:
areas.dropna(inplace=True)
areas.head(10)

,Neighborhood,Link,Latitude,Longitude
0,Abbey Hey,/wiki/Abbey_Hey,53.4652,-2.1875
2,Ancoats,/wiki/Ancoats,53.483333,-2.229722
3,Ardwick,/wiki/Ardwick,53.469167,-2.218611
4,Ardwick Green,/wiki/Ardwick_Green,53.4715278,-2.2245833
5,Baguley,/wiki/Baguley,53.3990,-2.2760
6,Barlow Moor,/wiki/Barlow_Moor,53.43,-2.26
7,"Belle Vue, Manchester","/wiki/Belle_Vue,_Manchester",53.4652,-2.1875
8,Benchill,/wiki/Benchill,53.38855,-2.26309
9,"Beswick, Manchester","/wiki/Beswick,_Manchester",53.474256,-2.202663
10,Blackley,/wiki/Blackley,53.52246,-2.22282


In [187]:
CLIENT_ID = '13XYAJUS4H2UNBY3TV35FZ5WIWFQ2LH2Z4G21PSGJHUWZYIH' # your Foursquare ID
CLIENT_SECRET = 'TR4NDYBCZERLHHGB0KEBZ5CC3ITKOLT3RO3TVCJD1F4PI1ZV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [188]:
LIMIT = 100
radius = 500 

In [189]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Exploring each area in Manchester 

In [193]:
areas_venues = getNearbyVenues(names=areas['Neighborhood'],
                                   latitudes=areas['Latitude'],
                                   longitudes=areas['Longitude']
                                  )


Abbey Hey
Ancoats
Ardwick
Ardwick Green
Baguley
Barlow Moor
Belle Vue, Manchester
Benchill
Beswick, Manchester
Blackley
Bradford-with-Beswick
Bradford, Manchester
Brooklands (Manchester ward)
Burnage
Burnage (ward)
Castlefield
Cheetham, Manchester
Chorlton (ward)
Chorlton Park (ward)
Chorlton-cum-Hardy
Chorlton-on-Medlock
Chorltonville
Clayton, Manchester
Collyhurst
Crumpsall
Didsbury
Fallowfield
Glenbrook, Greater Manchester
Gorton
Great Heaton
Green Quarter
Greenheys, Manchester
Harpurhey
Highfield Country Park
Hulme
Ladybarn
Levenshulme
Longsight
Manchester city centre
Merseybank
Miles Platting
Moss Side
Moston, Manchester
New Islington
New Moston
Newall Green
Newton Heath
Northenden
Northern Moor
Northern Quarter (Manchester)
Old Moat
Old Moat (ward)
Openshaw
Parrs Wood
Peel Hall, Wythenshawe
Ringway, Manchester
Sharston
Smedley, Manchester
Spinningfields
St John's, Manchester
Victoria Park, Manchester
West Gorton
Whalley Range, Manchester
Withington
Woodhouse Park
Wythenshawe


### Venues by neighborhood

In [194]:
areas_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Abbey Hey,7,7,7,7,7,7
Ancoats,48,48,48,48,48,48
Ardwick,8,8,8,8,8,8
Ardwick Green,4,4,4,4,4,4
Baguley,3,3,3,3,3,3
Barlow Moor,5,5,5,5,5,5
"Belle Vue, Manchester",7,7,7,7,7,7
Benchill,3,3,3,3,3,3
"Beswick, Manchester",4,4,4,4,4,4


In [196]:
print('There are {} uniques categories.'.format(len(areas_venues['Venue Category'].unique())))

There are 169 uniques categories.


In [198]:
areas_onehot = pd.get_dummies(areas_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
areas_onehot['Neighborhood'] = areas_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [areas_onehot.columns[-1]] + list(areas_onehot.columns[:-1])
areas_onehot = areas_onehot[fixed_columns]

areas_onehot.head()

,Neighborhood,Adult Boutique,Airport Food Court,Airport Lounge,Airport Service,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,...,Theater,Toy / Game Store,Track Stadium,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar
0,Abbey Hey,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Abbey Hey,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Abbey Hey,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Abbey Hey,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Abbey Hey,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [200]:
areas_grouped = areas_onehot.groupby('Neighborhood').mean().reset_index()
areas_grouped

,Neighborhood,Adult Boutique,Airport Food Court,Airport Lounge,Airport Service,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,...,Theater,Toy / Game Store,Track Stadium,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar
0,Abbey Hey,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Ancoats,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.041667,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.020833,0.000000,0.000000
2,Ardwick,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Ardwick Green,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Baguley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Barlow Moor,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,"Belle Vue, Manchester",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Benchill,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000
8,"Beswick, Manchester",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Blackley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [202]:
num_top_venues = 5

for hood in areas_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = areas_grouped[areas_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Abbey Hey----
            venue  freq
0       Racetrack  0.14
1   Track Stadium  0.14
2  Discount Store  0.14
3   Movie Theater  0.14
4            Park  0.14


----Ancoats----
                 venue  freq
0          Coffee Shop  0.15
1                  Bar  0.06
2             Tea Room  0.06
3          Record Shop  0.06
4  Arts & Crafts Store  0.04


----Ardwick----
               venue  freq
0     Sandwich Place  0.25
1  Convenience Store  0.12
2            Brewery  0.12
3      Train Station  0.12
4       Concert Hall  0.12


----Ardwick Green----
            venue  freq
0         Brewery  0.50
1    Camera Store  0.25
2    Concert Hall  0.25
3  Adult Boutique  0.00
4       Pet Store  0.00


----Baguley----
                        venue  freq
0                        Café  0.33
1  Construction & Landscaping  0.33
2                         Bar  0.33
3                    Pharmacy  0.00
4                   Nightclub  0.00


----Barlow Moor----
                     venue  freq
0        

                venue  freq
0       Grocery Store  0.25
1   Electronics Store  0.25
2  Chinese Restaurant  0.25
3            Bus Stop  0.25
4            Pharmacy  0.00


----Northern Quarter (Manchester)----
         venue  freq
0  Coffee Shop  0.10
1          Bar  0.06
2  Record Shop  0.04
3     Tea Room  0.04
4         Café  0.04


----Old Moat----
               venue  freq
0      Grocery Store  0.17
1                Pub  0.08
2  Convenience Store  0.08
3               Park  0.08
4  Fish & Chips Shop  0.08


----Old Moat (ward)----
               venue  freq
0      Grocery Store  0.17
1                Pub  0.08
2  Convenience Store  0.08
3               Park  0.08
4  Fish & Chips Shop  0.08


----Openshaw----
            venue  freq
0  Farmers Market  0.17
1     Pizza Place  0.17
2  Sandwich Place  0.17
3  Clothing Store  0.17
4   Go Kart Track  0.17


----Parrs Wood----
                  venue  freq
0                   Pub  0.12
1           Coffee Shop  0.12
2                  Café

Function to sort the venues in descending order

In [203]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [206]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
areas_venues_sorted = pd.DataFrame(columns=columns)
areas_venues_sorted['Neighborhood'] = areas_grouped['Neighborhood']

for ind in np.arange(areas_grouped.shape[0]):
    areas_venues_sorted.iloc[ind, 1:] = return_most_common_venues(areas_grouped.iloc[ind, :], num_top_venues)

areas_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abbey Hey,Bus Stop,Hotel,Discount Store,Racetrack,Track Stadium,Movie Theater,Park,Falafel Restaurant,Flea Market,Fish Market
1,Ancoats,Coffee Shop,Tea Room,Record Shop,Bar,Pub,Beer Bar,Arts & Crafts Store,Cocktail Bar,Fish & Chips Shop,Mexican Restaurant
2,Ardwick,Sandwich Place,Concert Hall,Train Station,Fast Food Restaurant,Brewery,Bakery,Convenience Store,Gym,Event Space,Fish & Chips Shop
3,Ardwick Green,Brewery,Concert Hall,Camera Store,Wine Bar,Farm,Food,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
4,Baguley,Bar,Construction & Landscaping,Café,Falafel Restaurant,Food,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
5,Barlow Moor,Hotel,Italian Restaurant,Furniture / Home Store,South Indian Restaurant,Indian Restaurant,Farm,Food,Flea Market,Fish Market,Fish & Chips Shop
6,"Belle Vue, Manchester",Bus Stop,Hotel,Discount Store,Racetrack,Track Stadium,Movie Theater,Park,Falafel Restaurant,Flea Market,Fish Market
7,Benchill,Print Shop,Tram Station,Coffee Shop,Entertainment Service,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
8,"Beswick, Manchester",Bus Stop,Gym,Grocery Store,Wine Bar,Fabric Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
9,Blackley,Pub,Grocery Store,Convenience Store,Bar,Home Service,Discount Store,Fabric Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant


### Clustering neigborhoods

In [213]:
# set number of clusters
kclusters = 5

areas_grouped_clustering = areas_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(areas_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:15] 

array([1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 2, 1], dtype=int32)

In [215]:
len(areas)

66

In [218]:
len(areas_venues_sorted)


66

In [221]:
kmeans.labels_

array([1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 2, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 4],
      dtype=int32)

In [224]:
# add clustering labels
#areas_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

areas_merged = areas

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
areas_merged = areas_merged.join(areas_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

areas_merged.head() # check the last columns!

,Neighborhood,Link,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abbey Hey,/wiki/Abbey_Hey,53.4652,-2.1875,1,Bus Stop,Hotel,Discount Store,Racetrack,Track Stadium,Movie Theater,Park,Falafel Restaurant,Flea Market,Fish Market
2,Ancoats,/wiki/Ancoats,53.483333,-2.229722,1,Coffee Shop,Tea Room,Record Shop,Bar,Pub,Beer Bar,Arts & Crafts Store,Cocktail Bar,Fish & Chips Shop,Mexican Restaurant
3,Ardwick,/wiki/Ardwick,53.469167,-2.218611,0,Sandwich Place,Concert Hall,Train Station,Fast Food Restaurant,Brewery,Bakery,Convenience Store,Gym,Event Space,Fish & Chips Shop
4,Ardwick Green,/wiki/Ardwick_Green,53.4715278,-2.2245833,1,Brewery,Concert Hall,Camera Store,Wine Bar,Farm,Food,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
5,Baguley,/wiki/Baguley,53.3990,-2.2760,1,Bar,Construction & Landscaping,Café,Falafel Restaurant,Food,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


In [236]:
areas_merged["Latitude"] = pd.to_numeric(areas_merged["Latitude"], downcast="float")
areas_merged["Longitude"] = pd.to_numeric(areas_merged["Longitude"], downcast="float")


In [237]:
latitude=53.483959
longitude=-2.244644

In [240]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(areas_merged['Latitude'], areas_merged['Longitude'], areas_merged['Neighborhood'], areas_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Cluster 0

In [249]:
areas_merged.loc[areas_merged['Cluster Labels'] == 0, areas_merged.columns[[0] + list(range(5, areas_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Ardwick,Sandwich Place,Concert Hall,Train Station,Fast Food Restaurant,Brewery,Bakery,Convenience Store,Gym,Event Space,Fish & Chips Shop
8,Benchill,Print Shop,Tram Station,Coffee Shop,Entertainment Service,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
11,Bradford-with-Beswick,Tram Station,Grocery Store,Child Care Service,Sandwich Place,Wine Bar,Event Space,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
17,"Cheetham, Manchester",Concert Hall,Museum,Soccer Field,Tram Station,Wine Bar,Fabric Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
26,Crumpsall,Coffee Shop,Sandwich Place,Tram Station,Asian Restaurant,Bakery,Park,Supermarket,Wine Bar,Fish Market,Fish & Chips Shop
59,"Smedley, Manchester",Moving Target,Racetrack,Tram Station,Art Gallery,Wine Bar,Fabric Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
67,Woodhouse Park,Fast Food Restaurant,Soccer Field,Sandwich Place,Discount Store,Warehouse Store,Tram Station,Coffee Shop,Fish & Chips Shop,Bus Station,Tanning Salon


Cluster 1

In [248]:
areas_merged.loc[areas_merged['Cluster Labels'] == 1, areas_merged.columns[[0] + list(range(5, areas_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abbey Hey,Bus Stop,Hotel,Discount Store,Racetrack,Track Stadium,Movie Theater,Park,Falafel Restaurant,Flea Market,Fish Market
2,Ancoats,Coffee Shop,Tea Room,Record Shop,Bar,Pub,Beer Bar,Arts & Crafts Store,Cocktail Bar,Fish & Chips Shop,Mexican Restaurant
4,Ardwick Green,Brewery,Concert Hall,Camera Store,Wine Bar,Farm,Food,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
5,Baguley,Bar,Construction & Landscaping,Café,Falafel Restaurant,Food,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
6,Barlow Moor,Hotel,Italian Restaurant,Furniture / Home Store,South Indian Restaurant,Indian Restaurant,Farm,Food,Flea Market,Fish Market,Fish & Chips Shop
7,"Belle Vue, Manchester",Bus Stop,Hotel,Discount Store,Racetrack,Track Stadium,Movie Theater,Park,Falafel Restaurant,Flea Market,Fish Market
9,"Beswick, Manchester",Bus Stop,Gym,Grocery Store,Wine Bar,Fabric Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
10,Blackley,Pub,Grocery Store,Convenience Store,Bar,Home Service,Discount Store,Fabric Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant
12,"Bradford, Manchester",Soccer Field,Café,Restaurant,Supermarket,Fish & Chips Shop,Sporting Goods Shop,Bar,Sandwich Place,Pub,Lounge
13,Brooklands (Manchester ward),Hotel,Convenience Store,Sports Club,Fabric Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


Cluster 2

In [250]:
areas_merged.loc[areas_merged['Cluster Labels'] == 2, areas_merged.columns[[0] + list(range(5, areas_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Burnage,Pub,Supermarket,Convenience Store,Harbor / Marina,Gym Pool,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
46,New Moston,Pub,Convenience Store,Fabric Shop,Food,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
56,"Peel Hall, Wythenshawe",Convenience Store,Dance Studio,Home Service,History Museum,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


Cluster 3

In [252]:
areas_merged.loc[areas_merged['Cluster Labels'] == 3, areas_merged.columns[[0] + list(range(5, areas_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,Harpurhey,Flea Market,Supermarket,Wine Bar,Food Court,Food,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


Cluster 4

In [251]:
areas_merged.loc[areas_merged['Cluster Labels'] == 4, areas_merged.columns[[0] + list(range(5, areas_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Wythenshawe,Tennis Court,Farm,Food Court,Food,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [ ]:
bands_df.to_excel('/Users/mamakino/Downloads/bands.xlsx')

In [ ]:
cities_gens_density.to_excel('/Users/mamakino/Downloads/cities_gens.xlsx')